# Question 2: CRUD Operations in SQL Server

Objective: \
Basic SQL Server interactions.

Task: \
Create an SQL table schema to store time-series metal prices. Include fields like Date, Metal, Price.
Demonstrate basic CRUD operations


## Solution:
For this task, I included all the available metal price time-series in the newly created database. Since the task requests the demonstration of basic CRUD operations but only specifies creating a table and updating it (‘C’ and ‘U’), I added two additional functions to perform the read and delete operations (‘R’ and ‘D’).

In [53]:
import os
import sqlite3

import pandas as pd
import numpy as np

### Parameters ###

data_path = f"{os.pardir}/data/MarketData.csv"

### Load the data and select the columns of interest ###

raw_df = pd.read_csv(data_path)
# Remove first 6 rows (headers)
df = raw_df.iloc[6:, :].copy()
# Get columns names for metals
columns_metals = raw_df.iloc[2, 1:-1].values
columns_metals = [col.split(" ")[1] for col in columns_metals]
print(f"Metals to include in the database: {columns_metals}")

### Create (or connect only) to the database and add the data ###

try:
    conn = sqlite3.connect(f"{os.pardir}/market_data.db")
    print("Connection to the database established.")
    cur = conn.cursor()
    # Create a table for metals
    cur.execute("""
        CREATE TABLE IF NOT EXISTS MetalPrices (
            Date DATE NOT NULL,
            Metal NVARCHAR(50) NOT NULL,
            Price DECIMAL(10, 2) NOT NULL,
            PRIMARY KEY (Date, Metal)
        );
    """)

    # Insert data into the table, metal by metal
    dates = pd.to_datetime(df.iloc[:, 0], dayfirst=True).dt.strftime("%Y-%m-%d")
    for m, metal in enumerate(columns_metals):
        prices = df.iloc[:, m + 1].values
        metal = [metal] * len(prices)
        data = np.column_stack((dates, metal, prices))
        cur.executemany(
            """
            INSERT OR REPLACE INTO MetalPrices (Date, Metal, Price)
            VALUES (?, ?, ?);
        """,
            data,
        )
    print("\tData inserted into the database.")

    conn.commit()
    cur.close()
    conn.close()

except sqlite3.Error as e:
    print(f"Error {e} occurred while connecting to the database")
finally:
    if conn:
        conn.close()
        print("Connection to the database closed.")


Metals to include in the database: ['COPPER', 'ALUMINUM', 'ZINC', 'LEAD', 'TIN']
Connection to the database established.
	Data inserted into the database.
Connection to the database closed.


In [54]:
def read_data(start_date, end_date, metal):
    """From start_date to end_date, returns the data from the database for specified metal.
    INPUTs:
    - start_date, end_date: str, YYYY-MM-DD
    - metal: str, name of the metal (e.g. 'COPPER')
    OUTPUT:
    - df: DataFrame, with columns ['Date', 'Metal', 'Price']
    """
    try:
        conn = sqlite3.connect(f"{os.pardir}/market_data.db")
        cur = conn.cursor()
        # Query the database
        query = """
            SELECT * FROM MetalPrices
            WHERE Date BETWEEN ? AND ? AND Metal = ?;
        """
        cur.execute(query, (start_date, end_date, metal))
        rows = cur.fetchall()
        # Get column names dynamically from the database schema
        column_query = "PRAGMA table_info(MetalPrices);"
        cur.execute(column_query)
        columns_info = cur.fetchall()
        column_names = [column[1] for column in columns_info]
        # Convert to DataFrame
        df = pd.DataFrame(rows, columns=column_names)
        cur.close()
    except sqlite3.Error as e:
        print(f"Error {e} occurred while reading from the database")
        df = None
    finally:
        if conn:
            conn.close()
    return df


def delete_data(date, metal):
    """Delete data from the database for a specific date and metal.
    INPUTs:
    - date: str, YYYY-MM-DD
    - metal: str, name of the metal (e.g. 'COPPER')
    """
    try:
        conn = sqlite3.connect(f"{os.pardir}/market_data.db")
        cur = conn.cursor()
        # Select the rows to be deleted
        query = """
            SELECT * FROM MetalPrices
            WHERE Date = ? AND Metal = ?;
        """
        cur.execute(query, (date, metal))
        rows_to_delete = cur.fetchall()
        print(f"\tRow found to be deleted: {rows_to_delete}.")
        # Delete the rows
        query = """
            DELETE FROM MetalPrices
            WHERE Date = ? AND Metal = ?;
        """
        cur.execute(query, (date, metal))
        conn.commit()
        cur.close()
    except sqlite3.Error as e:
        print(f"Error {e} occurred while deleting from the database")
    finally:
        if conn:
            conn.close()
    return None


print("Data inserted into the database:")
df = read_data("2010-01-01", "2021-01-01", "ALUMINUM")
print(df.head())

date_to_be_deleted = "2010-01-01"
print(f"\nDelete row for {date_to_be_deleted} - ALUMINUM.")
delete_data(date_to_be_deleted, "ALUMINUM")
print("\nData after deletion of a row:")
df = read_data("2010-01-01", "2021-01-01", "ALUMINUM")
print(df.head())

Data inserted into the database:
         Date     Metal   Price
0  2010-01-01  ALUMINUM  2230.0
1  2010-01-04  ALUMINUM  2267.0
2  2010-01-05  ALUMINUM  2302.0
3  2010-01-06  ALUMINUM  2377.0
4  2010-01-07  ALUMINUM  2310.0

Delete row for 2010-01-01 - ALUMINUM.
	Row found to be deleted: [('2010-01-01', 'ALUMINUM', 2230)].

Data after deletion of a row:
         Date     Metal   Price
0  2010-01-04  ALUMINUM  2267.0
1  2010-01-05  ALUMINUM  2302.0
2  2010-01-06  ALUMINUM  2377.0
3  2010-01-07  ALUMINUM  2310.0
4  2010-01-08  ALUMINUM  2284.0
